In [1]:
import torch
import random
import numpy as np
from functools import partial

In [2]:
# pip install torch torch_geometric torch-scatter pandas scikit-learn wandb
# python -m pip install gurobipy

In [3]:
from temp.data.gen import parallel_generate_problem, parallel_generate_solutions
from temp.data.problem import setcover
from temp.data.info import ModelInfo
from temp.data.dataset import ModelGraphDataset
from temp.data.aug import parallel_augment_info, augment_info


from temp.solver.utils import solve_inst


In [4]:
PRE_TRAIN_DIR = "temp/pre_train"
TRAIN_DIR = "temp/train"
VALID_DIR = "temp/valid"

In [5]:
# parallel_generate_problem(setcover, PRE_TRAIN_DIR, 100, 10)
# parallel_generate_solutions(PRE_TRAIN_DIR, 10)

# parallel_generate_problem(setcover, TRAIN_DIR, 10000, 10)
# parallel_generate_solutions(TRAIN_DIR, 10)

# parallel_generate_problem(setcover, VALID_DIR, 100, 10)
# parallel_generate_solutions(VALID_DIR, 10)

In [6]:
m = setcover()
vals = solve_inst(m)
info = ModelInfo.from_model(m)
info.var_info.sols = np.array([[m.objVal] + vals])

Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 22.4.0 22E252)

CPU model: Intel(R) Core(TM) i9-9980HK CPU @ 2.40GHz
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 128 rows, 256 columns and 3252 nonzeros
Model fingerprint: 0x115ddd59
Variable types: 0 continuous, 256 integer (256 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 182.0000000
Presolve time: 0.00s
Presolved: 128 rows, 256 columns, 3252 nonzeros
Variable types: 0 continuous, 256 integer (256 binary)

Root relaxation: objective 7.204726e+01, 214 iterations, 0.00 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Nod

In [7]:
# from temp.data.dataset import info_to_data
# d = info_to_data(info)
# aug = partial(parallel_augment_info, n=100)

In [ ]:
pre_train_d = ModelGraphDataset(PRE_TRAIN_DIR, augment=augment_info)
train_d = ModelGraphDataset(TRAIN_DIR, augment=augment_info)
valid_d = ModelGraphDataset(VALID_DIR)

data = pre_train_d[0][1]
var_feature_size = data.var_node_features.size(-1)
con_feature_size = data.con_node_features.size(-1) 

In [10]:
import pandas as pd
CFG = pd.read_excel("temp/trained_models/setcover_model_configs.xlsx", index_col=0).loc[0].to_dict()
CFG["num_epochs"] = 5
CFG["num_layers"] = 4
CFG["hidden"] = 8

In [11]:
from temp.model.utils import get_model
from temp.model.trainer import train

In [12]:
model_name, model, criterion, optimizer, scheduler = get_model(".", var_feature_size, con_feature_size, n_batches=1, **CFG)

In [13]:
scheduler.total_steps = 10000

In [14]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed()
    np.random.seed(worker_seed)
    random.seed(worker_seed)

from torch_geometric.loader import DataLoader
pretrain_loader = DataLoader(pre_train_d, batch_size=8, shuffle=True, worker_init_fn=seed_worker, generator=torch.Generator().manual_seed(0))
train_loader = DataLoader(train_d, batch_size=8, shuffle=True, worker_init_fn=seed_worker, generator=torch.Generator().manual_seed(0))
val_loader = DataLoader(valid_d, batch_size=8, shuffle=True, worker_init_fn=seed_worker, generator=torch.Generator().manual_seed(0))

In [15]:
scheduler.total_steps = 10000

In [16]:
train(model_name, model, criterion, optimizer, scheduler, pretrain_loader, train_loader, val_loader, CFG, False, "./")

>> Training starts on the current device cpu
>> Pretraining for prenorm...


  0%|          | 0/13 [00:00<?, ?it/s]


ValueError: too many values to unpack (expected 2)